<a href="https://colab.research.google.com/github/simnpeter/usernameApproving/blob/main/Low_level_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import pickle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path='/content/drive/My Drive/Colab Notebooks/dataset.csv'

def custom_parser(line):
    parts = line.split(";")  # Felosztjuk a sort pontosvessző mentén
    # Az első elem az összes többi elem
    username = parts[0] if len(parts) == 1 else ";".join(parts[:-1])
    label = int(parts[-1])  # Utolsó elem a címke, ami számmá konvertálható
    return username, label

data = []
with open(path, "r") as file:
  for line in file:
        username, label = custom_parser(line.strip())
        data.append((username, label))

        # Az adatok pandas DataFrame-be helyezése
df = pd.DataFrame(data, columns=["Username", "Label"])

label_counts = df['Label'].value_counts()

grouped = df.groupby('Label')

balanced_df = grouped.apply(lambda x: x.sample(n=label_counts.min()))

balanced_df = balanced_df.reset_index(drop=True)

# 2. Felosztás X és y változókra
X = df['Username']
y = df['Label']

X_balanced = balanced_df['Username']
y_balanced = balanced_df['Label']


vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)
X_balanced = vectorizer.fit_transform(X_balanced)


# 3. Tanító- és tesztadatokra felosztás
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_balanced_train, X_balanced_test, y_balanced_train, y_balanced_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)



In [ ]:
import pandas as pd

# Tegyük fel, hogy df a DataFrame, amiben van a 'username' és 'approved' oszlop
# Itt feltételezem, hogy a 'approved' oszlop értékei 0 vagy 1, ahol 1 az 'approved' és 0 a nem jóváhagyott felhasználót jelöli

# Készítsük el az instruction, input és output oszlopokat a DataFrame-ből
df['instruction'] = "Decide if the username is approvable or not"
df['input'] = df['Username']
df['output'] = df['Label'].apply(lambda x: "approved" if x == 1 else "not approved")

# Válaszd ki az instruction, input és output oszlopokat és konvertáld JSON formátumba
json_data = df[['instruction', 'input', 'output']].to_json(orient='records')

# Mentsd el a JSON adatokat egy fájlba
with open('/content/drive/My Drive/Colab Notebooks/dataset.json', 'w') as f:
    f.write(json_data)

print("JSON file successfully created!")


JSON file successfully created!


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")

    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [ ]:
svm_classifier = SVC(C=0.1, kernel='linear')
svm_classifier.fit(X_train, y_train)

with open('/content/drive/My Drive/Colab Notebooks/low_level_models/svm.pkl', 'wb') as f:
    pickle.dump(svm_classifier, f)

print_score(svm_classifier, X_train, y_train, X_test, y_test, train=True)
print_score(svm_classifier, X_train, y_train, X_test, y_test, train=False)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Result:
Accuracy Score: 89.74%
_______________________________________________
CLASSIFICATION REPORT:
                0            1  accuracy     macro avg  weighted avg
precision     0.0     0.897446  0.897446      0.448723      0.805409
recall        0.0     1.000000  0.897446      0.500000      0.897446
f1-score      0.0     0.945952  0.897446      0.472976      0.848940
support    1044.0  9136.000000  0.897446  10180.000000  10180.000000
_______________________________________________
Confusion Matrix: 
 [[   0 1044]
 [   0 9136]]

Test Result:
Accuracy Score: 89.98%
_______________________________________________
CLASSIFICATION REPORT:
               0            1  accuracy    macro avg  weighted avg
precision    0.0     0.899843  0.899843     0.449921      0.809717
recall       0.0     1.000000  0.899843     0.500000      0.899843
f1-score     0.0     0.947281  0.899843     0.473641      0.852404
support    255.0  2291.000000  0.899843  2546.000000   2546.000000
_________

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
svm_balanced_classifier = SVC(C=0.1, kernel='rbf')
svm_balanced_classifier.fit(X_balanced_train, y_balanced_train)

with open('/content/drive/My Drive/Colab Notebooks/low_level_models/svm_balanced.pkl', 'wb') as f:
    pickle.dump(svm_balanced_classifier, f)

print_score(svm_balanced_classifier, X_balanced_train, y_balanced_train, X_balanced_test, y_balanced_test, train=True)
print_score(svm_balanced_classifier, X_balanced_train, y_balanced_train, X_balanced_test, y_balanced_test, train=False)

Train Result:
Accuracy Score: 63.72%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.776892     0.592640  0.637151     0.684766      0.684145
recall        0.377907     0.892925  0.637151     0.635416      0.637151
f1-score      0.508475     0.712433  0.637151     0.610454      0.611141
support    1032.000000  1046.000000  0.637151  2078.000000   2078.000000
_______________________________________________
Confusion Matrix: 
 [[390 642]
 [112 934]]

Test Result:
Accuracy Score: 52.88%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy   macro avg  weighted avg
precision    0.696429    0.508621  0.528846    0.602525      0.605053
recall       0.146067    0.932806  0.528846    0.539437      0.528846
f1-score     0.241486    0.658298  0.528846    0.449892      0.444281
support    267.000000  253.000000  0.528846  520

In [ ]:
rf_classifier = RandomForestClassifier(max_depth=None, min_samples_leaf=4, min_samples_split=2, n_estimators=100)
rf_classifier.fit(X_train, y_train)

with open('/content/drive/My Drive/Colab Notebooks/low_level_models/rfc.pkl', 'wb') as f:
    pickle.dump(rf_classifier, f)

print_score(rf_classifier, X_train, y_train, X_test, y_test, train=True)
print_score(rf_classifier, X_train, y_train, X_test, y_test, train=False)

Train Result:
Accuracy Score: 89.74%
_______________________________________________
CLASSIFICATION REPORT:
                0            1  accuracy     macro avg  weighted avg
precision     0.0     0.897446  0.897446      0.448723      0.805409
recall        0.0     1.000000  0.897446      0.500000      0.897446
f1-score      0.0     0.945952  0.897446      0.472976      0.848940
support    1044.0  9136.000000  0.897446  10180.000000  10180.000000
_______________________________________________
Confusion Matrix: 
 [[   0 1044]
 [   0 9136]]

Test Result:
Accuracy Score: 89.98%
_______________________________________________
CLASSIFICATION REPORT:
               0            1  accuracy    macro avg  weighted avg
precision    0.0     0.899843  0.899843     0.449921      0.809717
recall       0.0     1.000000  0.899843     0.500000      0.899843
f1-score     0.0     0.947281  0.899843     0.473641      0.852404
support    255.0  2291.000000  0.899843  2546.000000   2546.000000
_________

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [ ]:
rf_balanced_classifier = RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=300)
rf_balanced_classifier.fit(X_balanced_train, y_balanced_train)

with open('/content/drive/My Drive/Colab Notebooks/low_level_models/rfc_balanced.pkl', 'wb') as f:
    pickle.dump(rf_balanced_classifier, f)

print_score(rf_balanced_classifier, X_balanced_train, y_balanced_train, X_balanced_test, y_balanced_test, train=True)
print_score(rf_balanced_classifier, X_balanced_train, y_balanced_train, X_balanced_test, y_balanced_test, train=False)

Train Result:
Accuracy Score: 98.65%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.993124     0.980189  0.986526     0.986656      0.986613
recall        0.979651     0.993308  0.986526     0.986480      0.986526
f1-score      0.986341     0.986705  0.986526     0.986523      0.986524
support    1032.000000  1046.000000  0.986526  2078.000000   2078.000000
_______________________________________________
Confusion Matrix: 
 [[1011   21]
 [   7 1039]]

Test Result:
Accuracy Score: 52.12%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy   macro avg  weighted avg
precision    0.625000    0.504464  0.521154    0.564732      0.566355
recall       0.168539    0.893281  0.521154    0.530910      0.521154
f1-score     0.265487    0.644793  0.521154    0.455140      0.450034
support    267.000000  253.000000  0.521154 

In [ ]:
mlp_classifier = MLPClassifier(activation='relu', alpha= 0.0001, learning_rate='constant', solver='sgd')
mlp_classifier.fit(X_train, y_train)

with open('/content/drive/My Drive/Colab Notebooks/low_level_models/mlp.pkl', 'wb') as f:
    pickle.dump(mlp_classifier, f)

print_score(rf_classifier, X_train, y_train, X_test, y_test, train=True)
print_score(rf_classifier, X_train, y_train, X_test, y_test, train=False)

Train Result:
Accuracy Score: 89.74%
_______________________________________________
CLASSIFICATION REPORT:
                0            1  accuracy     macro avg  weighted avg
precision     0.0     0.897446  0.897446      0.448723      0.805409
recall        0.0     1.000000  0.897446      0.500000      0.897446
f1-score      0.0     0.945952  0.897446      0.472976      0.848940
support    1044.0  9136.000000  0.897446  10180.000000  10180.000000
_______________________________________________
Confusion Matrix: 
 [[   0 1044]
 [   0 9136]]

Test Result:
Accuracy Score: 89.98%
_______________________________________________
CLASSIFICATION REPORT:
               0            1  accuracy    macro avg  weighted avg
precision    0.0     0.899843  0.899843     0.449921      0.809717
recall       0.0     1.000000  0.899843     0.500000      0.899843
f1-score     0.0     0.947281  0.899843     0.473641      0.852404
support    255.0  2291.000000  0.899843  2546.000000   2546.000000
_________

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [ ]:
mlp_balanced_classifier = MLPClassifier(activation='relu', alpha=0.0001, learning_rate='constant', solver='adam')
mlp_balanced_classifier.fit(X_balanced_train, y_balanced_train)

with open('/content/drive/My Drive/Colab Notebooks/low_level_models/mlp_balanced.pkl', 'wb') as f:
    pickle.dump(mlp_balanced_classifier, f)

print_score(rf_balanced_classifier, X_balanced_train, y_balanced_train, X_balanced_test, y_balanced_test, train=True)
print_score(rf_balanced_classifier, X_balanced_train, y_balanced_train, X_balanced_test, y_balanced_test, train=False)

Train Result:
Accuracy Score: 98.65%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.993124     0.980189  0.986526     0.986656      0.986613
recall        0.979651     0.993308  0.986526     0.986480      0.986526
f1-score      0.986341     0.986705  0.986526     0.986523      0.986524
support    1032.000000  1046.000000  0.986526  2078.000000   2078.000000
_______________________________________________
Confusion Matrix: 
 [[1011   21]
 [   7 1039]]

Test Result:
Accuracy Score: 52.12%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy   macro avg  weighted avg
precision    0.625000    0.504464  0.521154    0.564732      0.566355
recall       0.168539    0.893281  0.521154    0.530910      0.521154
f1-score     0.265487    0.644793  0.521154    0.455140      0.450034
support    267.000000  253.000000  0.521154 